In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [8]:
# Define selected classes
selected_classes = ["AnnualCrop", "Residential"]

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Create label mapping (AnnualCrop → 0, Residential → 1)
class_to_index = full_dataset.class_to_idx  # {'AnnualCrop': 0, 'Forest': 1, ..., 'Residential': 7}
label_mapping = {class_to_index["AnnualCrop"]: 0, class_to_index["Residential"]: 1}

# Filter dataset and remap labels
filtered_data = [(img, label_mapping[label]) for img, label in full_dataset if label in label_mapping]

# Convert to DataLoader
data_loader = DataLoader(filtered_data, batch_size=32, shuffle=True)

print(f"Filtered dataset size: {len(filtered_data)} images")


Filtered dataset size: 6000 images


In [6]:
# Define CNN Model
class CNNModel(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [9]:
# Define model with 2 output classes
model = CNNModel(num_classes=2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(data_loader):.4f}")


Epoch [1/10], Loss: 0.1076
Epoch [2/10], Loss: 0.0219
Epoch [3/10], Loss: 0.0209
Epoch [4/10], Loss: 0.0063
Epoch [5/10], Loss: 0.0149
Epoch [6/10], Loss: 0.0051
Epoch [7/10], Loss: 0.0027
Epoch [8/10], Loss: 0.0032
Epoch [9/10], Loss: 0.0041
Epoch [10/10], Loss: 0.0057


In [10]:
# Save the model
torch.save(model.state_dict(), "eurosat_cnn_2.pth")
print("Model training complete and saved as eurosat_cnn_2.pth")

Model training complete and saved as eurosat_cnn_2.pth
